# Road Following - モデルの学習(resnet18モデルの学習)

このnotebookは、入力画像を読み込み、ターゲットに対応するx、y値のセットを出力するようにニューラルネットワークを学習します。

road followingではtorchvisionで用意されているResNet18モデルを少し変更して使います。

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### Create Dataset Instance(データセットインスタンスの作成)

[torch.utils.data.DataLoader](https://github.com/pytorch/pytorch/blob/master/torch/utils/data/dataloader.py)クラスを継承して``__len__`` と ``__getitem__``関数を実装実装した``XYDataset``クラスを作成します。このクラスは、画像をロードするための役割と、画像ファイル名からx,y値の値をパースして取得します。[torch.utils.data.DataLoader](https://github.com/pytorch/pytorch/blob/master/torch/utils/data/dataloader.py)を継承する事で、すべてのtorchデータユーティリティを使用する事ができます。

いくつかの変換（カラージッターなど）をデータセットにハードコーディングしました。カラージッターは画像の明るさ、コントラスト、彩度をランダムに変更します。\
ランダムに水平反転を有効にするオプション``random_hflips``を付けました。デフォルトは有効にしています。水平反転は「右車線にとどまる」必要が無い場合、つまり左右どちらの車線を通ってもいい場合に有効にすることでデータセットの特性が変わります。

In [ ]:
def get_x(path):
    """Gets the x value from the image filename"""
    return (float(int(path[3:6])) - 50.0) / 50.0

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path[7:10])) - 50.0) / 50.0

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.image_paths = glob.glob(os.path.join(directory, '*.jpg'))
        self.random_hflips = random_hflips
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        # ランダムに画像を水平反転する時、出力のxも対応するように反転する
        if self.random_hflips:
            if float(np.random.rand(1)) > 0.5:
                image = transforms.functional.hflip(image)
                x = -x

        # 画像をモデル学習の入力用データフォーマットに変換する
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        # ImageNetの正規化と同じパラメータでデータを正規化する
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=True)

### Split dataset into train and test sets(トレーニングデータとテストデータに分ける)
次に、データセットを*トレーニング用*と*テスト用*のデータセットに分割します。この例では、*トレーニング用*に90%, *テスト用*に10%で分けます。*テスト用*のデータセットは、学習中にモデルの精度を検証するために使用されます。

In [ ]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### Create data loaders to load data in batches(バッチ処理で学習データとテストデータを読み込むためのデータローダーを作成)

[torch.utils.data.DataLoader](https://github.com/pytorch/pytorch/blob/master/torch/utils/data/dataloader.py)クラスは、モデル学習中に次のデータ処理が完了出来るようにサブプロセスで並列処理にして実装します。\
データのシャッフル、バッチでのデータロードのために使用します。この例では、1回のバッチ処理で8枚の画像を使用します。これをバッチサイズと呼び、GPUのメモリ使用量と、モデルの精度に影響を与えます。

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

### Define Neural Network Model (JetBot用にモデルを変更する)

torchvisionで使用可能なImageNetデータセットで学習済みのResNet18モデルを使用します。

*転移学習*と呼ばれる手法で、すでに画像分類できる特徴を持つニューラルネットワーク層を、別の目的のために作られたモデルに適用することで、短時間で良好な結果を得られるモデルを作成することができます。

ResNet18の詳細: https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

転移学習の詳細：https://www.youtube.com/watch?v=yofjFQddwHE

In [ ]:
model = models.resnet18(pretrained=True)

ResNet18モデルはImageNetを学習するために作られているため、1000種類の画像分類が可能な出力を持っています。ResNet18モデル構造の全結合層(fully connected layer)を入れ替えて、JetBotで欲しい出力x,yの2種類を得られるモデル構造にします。

In [ ]:
print(model.fc.in_features)
model.fc = torch.nn.Linear(model.fc.in_features, 2)

デフォルトではモデルのweightはCPUで処理されるため、GPUを利用するようにモデルを設定します。

In [ ]:
print("before: {}".format(model.fc.weight.type()))
device = torch.device('cuda')
model = model.to(device)
print("after: {}".format(model.fc.weight.type()))

# Fine-tuning(ファインチューニング)
初めての学習では、この項目はスキップして**「視覚化ユーティリティ」**に進んでください。\
学習データが500件を超えて、学習時間が遅くなってきた時にファインチューニングが役立ちます。

学習データが800枚にもなると、学習に必要なメモリ容量が増えてきてSWAPが多く消費されるため、学習速度が大幅に低下してしまいます。\
この時、再学習するネットワーク層を限定することで学習時のメモリ使用量を減らしながら、高速に学習を進めることが可能となります。\
全結合層(`fully-connected layer`)だけを再学習しても、なかなか成果につながりにくいため、ここでは[ResNet18](https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py)モデルの`fc`と`layer4`を再学習させることにします。

In [ ]:
# モデルを凍結して使う場合は、全てのパラメータが持つ学習フラグを無効化します。デフォルトはrequires_grad = Trueで全てのパラメータを再学習します。
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# 特定のネットワーク層の凍結を解除して再学習させるために、パラメータ名を確認します。
for name, param in model.named_parameters():
    print(name)

全結合層(`fc`)と`layer4`を再学習させます。

In [ ]:
# fcとlayer4を再学習させます。fcは入れ替えているので必ず学習させてください。
for name, param in model.named_parameters():
    if name.startswith("layer4"):
        param.requires_grad = True
    if name.startswith("fc"):
        param.requires_grad = True

In [ ]:
# 再学習させるパラメータを確認します。
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

このように、特定の層を選んで再学習させることもできます。

# 視覚化ユーティリティ
[bokeh](https://docs.bokeh.org/en/latest/docs/installation.html)を使って学習中の損失(loss)と精度(accuracy)をグラフに表示することができます。

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models.tickers import SingleIntervalTicker
output_notebook()

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

p1 = figure(title="Loss", x_axis_label="Epoch", plot_height=300, plot_width=360)
p2 = figure(title="Accuracy", x_axis_label="Epoch", plot_height=300, plot_width=360)

source1 = ColumnDataSource(data={'epochs': [], 'trainlosses': [], 'testlosses': [] })
source2 = ColumnDataSource(data={'epochs': [], 'train_accuracies': [], 'test_accuracies': []})

#r = p1.multi_line(ys=['trainlosses', 'testlosses'], xs='epochs', color=colors, alpha=0.8, legend_label=['Training','Test'], source=source)
r1 = p1.line(x='epochs', y='trainlosses', line_width=2, color=colors[0], alpha=0.8, legend_label="Train", source=source1)
r2 = p1.line(x='epochs', y='testlosses', line_width=2, color=colors[1], alpha=0.8, legend_label="Test", source=source1)

r3 = p2.line(x='epochs', y='train_accuracies', line_width=2, color=colors[0], alpha=0.8, legend_label="Train", source=source2)
r4 = p2.line(x='epochs', y='test_accuracies', line_width=2, color=colors[1], alpha=0.8, legend_label="Test", source=source2)

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p2.legend.location = "bottom_right"
p2.legend.click_policy="hide"

### Training(モデルの学習)

20エポック学習し、各エポックで以前の最高精度と現在の精度を比較することにより、最高精度を更新した場合に保存します。\
現在の精度が以前の最高精度と等しい場合は、損失の少ない方を保存します。

> 1エポックは、私たちが用意したトレーニング用のデータ全部を1回学習することです。一度に8枚の画像を学習するミニバッチ処理を複数回実行することで1エポックが完了します。

**Collision Avoidance**の時は、「free(直進する)」or「blocked(旋回する)」それぞれに対する正解ラベルは`True`or`False`の 0 or 1 で定義できました。また、「free(直進する)」or「blocked(旋回する)」のうち、1つだけが`True`となるため**one hot value**として定義できました。**one hot value**を予測する場合、モデルの精度の定義はテストデータでの予測結果が、**「正解ラベルと一致している件数」÷「テストデータの総数」**となります。

**Road Following**では正解ラベルはx,yの2出力それぞれ[-1.0,1.0]のfloat型の範囲になります。このため、正解ラベルと一致しない場合が多くなり、精度の定義は難しくなります。この場合、最低損失を更新した場合にモデルを保存します。

今回は、Collision Avoidanceと同じような学習コードの構成にしたいので、何らかの方法で精度を定義したいとおもいます。\
ここでは`MSE`をlossとして定義しているため、`1.0 - RMSE`をaccuracyとして定義することにします。

one hot valueの詳細：https://www.youtube.com/watch?v=v_4KWmkwmsU \
回帰モデルの評価指標の詳細：https://www.ritchieng.com/machine-learning-evaluate-linear-regression-model/#15.-Model-Evaluation-Metrics-for-Regression

In [ ]:
NUM_EPOCHS = 20
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_accuracy = 0.0
saved_loss = 1e9

optimizer = optim.Adam(model.parameters())

handle = show(row(p1, p2), notebook_handle=True)

# create RMSELoss function
def RMSELoss(yhat,y):
    '''
    yhat: predicted value
    y: observed value
    '''
    return torch.sqrt(torch.mean((yhat-y)**2))

criterion = RMSELoss

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    train_error_count = 0.0 # for graph
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        train_error_count += criterion(outputs, labels) # for graph
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    test_error_count = 0.0 # for graph
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
        test_error_count += criterion(outputs, labels) # for graph
    test_loss /= len(test_loader)
    
    train_accuracy = 1.0 - float(train_error_count) / float(len(train_dataset)) # for graph
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset)) # for graph

    # 今回のepoch学習のテスト結果がよければ保存します
    is_saved = False
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy
        saved_loss = test_loss
        is_saved = True
    elif test_accuracy == best_accuracy:
        if test_loss < saved_loss:
            torch.save(model.state_dict(), BEST_MODEL_PATH)
            saved_loss = test_loss
            is_saved = True

    print('%d: %f, %f, %f, %f, ' % (epoch+1, train_loss, test_loss, train_accuracy, test_accuracy)+("saved" if is_saved else "not saved"))

    # 学習状況をグラフに表示します
    new_data1 = {'epochs': [epoch+1],
                 'trainlosses': [float(train_loss)],
                 'testlosses': [float(test_loss)] }
    source1.stream(new_data1)
    new_data2 = {'epochs': [epoch+1],
                 'train_accuracies': [float(train_accuracy)],
                 'test_accuracies': [float(test_accuracy)] }
    source2.stream(new_data2)
    push_notebook(handle=handle)

学習が完了すると、``live_demo.ipynb``で推論に使う``best_steering_model_xy.pth``が生成されます。

## Next(次)

次は、``live_demo.ipynb``を実行し、学習済みモデルで自動走行します。\
ノートブックメニューから`Kernel`->`Restert Kernel`を選んでJupyter kernelを再起動するか、JetBotを一度再起動してから次に進むとスムーズに進行できます。

TensorRTを試したい人は、trtフォルダの中の``convert_to_trt.ipynb``を実行し、学習済みモデルをTensorRT形式に変換し、``live_demo_trt.ipynb``を実行し自動走行します。